In [1]:
!ls ../models_library/raw_models/

DunedinPoAm.csv			 McCartneyBlood_2018_BMI.csv
EpiTOC2.csv			 McCartneyBlood_2018_Education.csv
HannumBlood_2013.csv		 McCartneyBlood_2018_HDL.csv
HannumBreast_2013.csv		 McCartneyBlood_2018_LDL.csv
HannumKidney_2013.csv		 McCartneyBlood_2018_Smoking.csv
HannumLung_2013.csv		 McCartneyBlood_2018_TC.csv
HorvathMultiShrunken_2013.csv	 McCartneyBlood_2018_TotalFat.csv
HorvathMulti_2013.csv		 McCartneyBlood_2018_WaistTipRatio.csv
KnightBlood_2016.csv		 PhenoAge.csv
LinBlood3CpG_2016.csv		 YingAdaptAge.csv
LinBlood99CpG_2016.csv		 YingCausAge.csv
McCartneyBlood_2018_Alcohol.csv  YingDamAge.csv


In [1]:
import numpy as np
import pandas as pd 


In [4]:
import sys
import os


data_library = os.path.abspath('../data_library')
sys.path.append(data_library)

from data_repository import import_data




In [3]:
! ls ../data_library/

__init__.py  __pycache__  data_repository.py  data_test.csv


In [4]:
SAMPLE_ID = 'GSE41169_series_matrix.txt'
URL = 'https://drive.google.com/file/d/1cZca-FHnfeUerXJOzqhnH34mUYjNOgba/view?usp=sharing'
data_test = import_data(SAMPLE_ID, URL)


Downloading...
From (original): https://drive.google.com/uc?id=1cZca-FHnfeUerXJOzqhnH34mUYjNOgba
From (redirected): https://drive.google.com/uc?id=1cZca-FHnfeUerXJOzqhnH34mUYjNOgba&confirm=t&uuid=f158dd4f-dbdb-4828-8b51-c6450a5612c9
To: /home/aryuna/bioinf_inst/project/ComputAge/computage/deage/GSE41169_series_matrix.txt
100%|██████████| 477M/477M [00:41<00:00, 11.5MB/s] 


In [5]:
data_test.to_csv('../data_library/data_test.csv', sep = '\t', index=False)


In [7]:
model_filepath = '../models_library/raw_models/HannumLung_2013.csv'

model_data = pd.read_csv(model_filepath)
features = model_data[['Feature_ID']]
coefficients = model_data[['Coef']]


In [28]:
df = data_test.copy()
df.columns = df.iloc[0]
df.columns = df.columns.str.replace('"', '')
df = df[1:]
df = df.reset_index(drop=True)
df.ID_REF = df.ID_REF.str.replace('"', '')


In [38]:
samples = list(df.columns)[1:]

In [42]:
df[samples] = df[samples].apply(pd.to_numeric)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485577 entries, 0 to 485576
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID_REF      485577 non-null  object 
 1   GSM1009660  485577 non-null  float64
 2   GSM1009661  485577 non-null  float64
dtypes: float64(2), object(1)
memory usage: 11.1+ MB


In [44]:
df_merged = df.merge(model_data, left_on='ID_REF', right_on='Feature_ID', how='right')

In [45]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID_REF      14 non-null     object 
 1   GSM1009660  14 non-null     float64
 2   GSM1009661  14 non-null     float64
 3   Feature_ID  14 non-null     object 
 4   Coef        14 non-null     float64
 5   Coord_hg19  14 non-null     object 
dtypes: float64(3), object(3)
memory usage: 804.0+ bytes


In [49]:
vectors = np.array(df_merged[samples].to_numpy())

In [71]:
coefficients

,Coef
0,-3.440
1,-22.800
2,48.800
3,8.500
4,3.950
5,40.300
6,-63.500
7,22.700
8,-0.885
9,24.800


In [65]:
features_array = np.array(coefficients)

In [55]:
vectors.shape

(14, 2)

In [60]:
vectors[:, 0]

array([0.09197747, 0.3159195 , 0.5040356 , 0.05984634, 0.1030878 ,
       0.01129568, 0.7679211 , 0.07017164, 0.04656127, 0.01154639,
       0.05297833, 0.158752  , 0.2492757 , 0.03991627])

In [58]:
vectors.transpose()

array([[0.09197747, 0.3159195 , 0.5040356 , 0.05984634, 0.1030878 ,
        0.01129568, 0.7679211 , 0.07017164, 0.04656127, 0.01154639,
        0.05297833, 0.158752  , 0.2492757 , 0.03991627],
       [0.06034835, 0.2680101 , 0.4410885 , 0.05883182, 0.07369265,
        0.00422823, 0.9013712 , 0.04339134, 0.02392316, 0.00620221,
        0.03814023, 0.1065092 , 0.1649368 , 0.02257136]])

In [68]:
features_array.transpose()

array([[ -3.44 , -22.8  ,  48.8  ,   8.5  ,   3.95 ,  40.3  , -63.5  ,
         22.7  ,  -0.885,  24.8  ,  -0.863,  50.9  ,  -3.75 ,  44.   ]])

In [61]:
vectors

array([[0.09197747, 0.06034835],
       [0.3159195 , 0.2680101 ],
       [0.5040356 , 0.4410885 ],
       [0.05984634, 0.05883182],
       [0.1030878 , 0.07369265],
       [0.01129568, 0.00422823],
       [0.7679211 , 0.9013712 ],
       [0.07017164, 0.04339134],
       [0.04656127, 0.02392316],
       [0.01154639, 0.00620221],
       [0.05297833, 0.03814023],
       [0.158752  , 0.1065092 ],
       [0.2492757 , 0.1649368 ],
       [0.03991627, 0.02257136]])

In [113]:
prediction = np.matmul( features_array.transpose(), vectors)

In [86]:
np.array(samples).shape

(2,)

In [89]:
dict(zip(samples, list(prediction)))

{'GSM1009660': array([-19.61998458, -34.18797063])}

In [102]:
for a, b in zip(samples, np.transpose(prediction)):
    print(a)
    l = b.item()
    print(l)

GSM1009660
-19.619984580540006
GSM1009661
-34.187970631690014


In [82]:
{A: B for A, B in zip(np.array(samples), prediction)}

{'GSM1009660': array([-19.61998458, -34.18797063])}

In [108]:
pred_dict = {}
for a, b in zip(samples, np.transpose(prediction)):
    pred_dict.update({a:b.item()})

pd.DataFrame.from_dict(pred_dict, orient='index', columns = ['prediction'])


,prediction
GSM1009660,-19.619985
GSM1009661,-34.187971


In [114]:
pd_prediction = pd.DataFrame()
pd_prediction['sample'] = samples
pd_prediction['predictions'] = prediction.transpose()

pd_prediction

,sample,predictions
0,GSM1009660,-19.619985
1,GSM1009661,-34.187971


In [122]:
df.to_csv('../data_library/data_test.csv', sep='\t', index=False)

In [5]:
df = pd.read_csv('../data_library/data_test.csv', sep = '\t')

In [7]:
df.head()

,ID_REF,GSM1009660,GSM1009661
0,cg00000029,0.454931,0.410287
1,cg00000108,0.907883,0.915894
2,cg00000109,0.862772,0.881000
3,cg00000165,0.160446,0.137600
4,cg00000236,0.728180,0.736708


In [8]:
from base import LinearMethylationModel

In [10]:
model_filepath = '../models_library/raw_models/HannumLung_2013.csv'

In [11]:
my_model = LinearMethylationModel(model_file_path=model_filepath)

predictions = my_model.predict(df)

In [12]:
predictions

,sample,predictions
0,GSM1009660,-19.619985
1,GSM1009661,-34.187971
